In [0]:
import requests
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import boto3
from io import BytesIO
from datetime import datetime
 import os

# 1. Buscar dados da API da NASA
url = "https://api.nasa.gov/insight_weather/?api_key=ywTPslt9SJHevhcwYCBGOCLTVjG1K6FsHRRjJksJ&feedtype=json&ver=1.0"
response = requests.get(url)
json_data = response.json()

# 2. Extrair os dias (sols) e montar os registros
sols = json_data.get('sol_keys', [])
records = []

for sol in sols:
    data = json_data.get(sol, {})
    record = {
        'sol': sol,
        'season': data.get('Season'),
        'first_utc': data.get('First_UTC'),
        'last_utc': data.get('Last_UTC'),
        'temp_avg': data.get('AT', {}).get('av')  # temperatura média
    }
    records.append(record)

# 3. Criar DataFrame
df = pd.DataFrame(records)
print("📊 DataFrame criado com", len(df), "linhas")

# 4. Converter para Parquet em memória
buffer = BytesIO()
table = pa.Table.from_pandas(df)
pq.write_table(table, buffer)
print("📦 Tamanho do arquivo Parquet:", buffer.getbuffer().nbytes, "bytes")

# 5. Enviar para o S3 na camada Bronze
bucket_name = "nasa-weather-bronze"
data_str = datetime.now().strftime("%Y-%m-%d")
object_key = f"bronze/nasa_weather/nasa_insight_{data_str}.parquet"

# 6. Cliente S3 com credenciais explícitas (temporário)
s3 = boto3.client(
    's3',
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")

    region_name='us-east-1'
)

# 7. Upload com tratamento de erro
try:
    s3.put_object(Bucket=bucket_name, Key=object_key, Body=buffer.getvalue())
    print(f"✅ Arquivo enviado para S3: {object_key}")
except Exception as e:
    print("❌ Erro ao enviar para o S3:", e)

